# Process WFABBA FIgLib Join

<b>Summary:</b><br>
Reads in parsed WFABBA data from csv files created from 0_get_wfabba.ipynb (WFABBA GOES-16 & WFABBA GOES-17 detections), smokeynet_test.json & smokeynet_valid.json (SmokeyNet predictions), and camera_metadata_hpwren.csv (contains locations of camera stations associated with SmokeyNet predictions). Join SmokeyNet detections with camera metadata to associate coordinates with every SmokeyNet prediction. For every camera station, join SmokeyNet predictions with potential WFABBA GOES-16 and WFABBA GOES-17 detections, then output results to csv files.<br>

- Read in parsed WFABBA data (outputted from 0_get_wfabba.ipynb), SmokeyNet predictions, and camera metadata.
- Join SmokeyNet predictions with potential WFABBA GOES-16/GOES-17 detections.
- Output results to csv files

<b>Output:</b><br>
../..<br>
└── data<br>
&emsp;&emsp;&emsp;└── processed<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── \<CAMERA_STATION_NAME\>_all_hard_voting_35.csv<br>

<b>Instructions:</b><br>
- See README.md in directory

<b>Areas for Improvement:</b><br>
Need to further look into approaches to join SmokeyNet detections with WFABBA GOES-16/GOES-17 detections. Currently looking at joining SmokeyNet predictions with WFABBA detections by location proximity distances (default of 35 miles), camera direction, and whether the detections happen at the exact minute. May need to look into temporal joins if going with current implementation of joins. 
If considering another join approach instead: Currently considering each image as an independent event. May need to consider groupings of images as an event instead. Consider finding first instance of SmokeyNet, WFABBA GOES-16, WFABBA GOES-17 detections?

In [1]:
import pandas as pd
import urllib.request
import datetime as dt
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from haversine import haversine, Unit
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import pytz
from sklearn.metrics import accuracy_score
import numpy as np
pd.set_option('display.max_columns', None)

## 1) Read in WFABBA Data and Consolidate into WFABBA GOES-16/GOES-17

In [2]:
# definte the processed and raw data directories
processed_data_dir = "../../data/processed/wfabba/"
raw_data_dir = "../../data/raw/"

In [3]:
# read in GOES 16 inputs
wfabba_goes_16_2019_df = pd.read_csv(processed_data_dir + "GOES-16-2019.csv")
wfabba_goes_16_2020_df = pd.read_csv(processed_data_dir + "GOES-16-2020.csv")
wfabba_goes_16_jan_2021_df = pd.read_csv(processed_data_dir + "GOES-16-Jan-2021.csv")
wfabba_goes_16_2021_df = pd.read_csv(processed_data_dir + "GOES-16-2021.csv")

In [4]:
# get rid of unnecessary columns including ones which contain the same values or all NaN
wfabba_goes_16_2019_df = wfabba_goes_16_2019_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_16_2020_df = wfabba_goes_16_2020_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_16_jan_2021_df = wfabba_goes_16_jan_2021_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_16_2021_df = wfabba_goes_16_2021_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"]) #2021 detections

In [5]:
# filter out any January data in wfabba_goes_16_2021_df since it already exists in wfabba_goes_16_jan_2021_df
print(len(wfabba_goes_16_2021_df))
wfabba_goes_16_2021_df = wfabba_goes_16_2021_df[wfabba_goes_16_2021_df["Timestamp"] >= "2021-02-01"]
wfabba_goes_16_2021_df = wfabba_goes_16_2021_df.reset_index()
wfabba_goes_16_2021_df = wfabba_goes_16_2021_df.drop(columns=["index"])
print(len(wfabba_goes_16_2021_df))
wfabba_goes_16_2021_df

94913
93868


,Version,Timestamp,Satellite,FlightModel,ScanMode,ProductType,FileName,MissingValueCode,Latitude,Longitude,Code,FRP,Fire Size,Fire Temp,Pixel Size,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco
0,6_6_001g,2021-02-01 23:41:11,GOES-16,FM?,GOES-16,GOES-16,f2021032234111.rev.6_6_001g.FDCC.GOES-16.txt,-9999,34.8464,-120.0688,15,31.3,-9999.0,-9999.0,10113770,292.657,272.489,286.602,282.154,71.593,62.486,-9999,91
1,6_6_001g,2021-02-01 23:41:11,GOES-16,FM?,GOES-16,GOES-16,f2021032234111.rev.6_6_001g.FDCC.GOES-16.txt,-9999,34.5600,-119.9001,15,25.7,-9999.0,-9999.0,10005460,292.216,273.886,287.452,282.747,71.528,62.199,-9999,91
2,6_6_001g,2021-02-01 23:40:14,GOES-16,FM?,GOES-16,GOES-16,f2021032234014.rev.6_6_001g.FDCF.GOES-16.txt,-9999,34.8464,-120.0688,15,32.7,-9999.0,-9999.0,10113154,292.929,271.184,286.603,282.059,71.431,62.486,-9999,91
3,6_6_001g,2021-02-01 23:46:11,GOES-16,FM?,GOES-16,GOES-16,f2021032234611.rev.6_6_001g.FDCC.GOES-16.txt,-9999,34.8464,-120.0688,15,31.4,-9999.0,-9999.0,10113770,292.547,269.709,286.453,282.579,72.410,62.486,-9999,91
4,6_6_001g,2021-02-01 23:21:11,GOES-16,FM?,GOES-16,GOES-16,f2021032232111.rev.6_6_001g.FDCC.GOES-16.txt,-9999,34.8464,-120.0688,15,22.7,-9999.0,-9999.0,10113770,292.327,272.043,287.418,281.058,68.429,62.486,-9999,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93863,6_6_001g,2022-02-09 09:40:20,GOES-16,FM?,GOES-16,GOES-16,f2022040094020.rev.6_6_001g.FDCF.GOES-16.txt,-9999,33.8289,-117.8737,10,59.3,92471.0,447.4,9326390,296.811,285.730,283.625,283.939,151.404,60.212,-9999,1
93864,6_6_001g,2022-02-09 09:40:20,GOES-16,FM?,GOES-16,GOES-16,f2022040094020.rev.6_6_001g.FDCF.GOES-16.txt,-9999,33.8030,-117.8540,10,68.2,269858.0,402.7,9316094,298.369,287.315,283.652,283.960,151.405,60.182,-9999,1
93865,6_6_001g,2022-02-09 10:01:17,GOES-16,FM?,GOES-16,GOES-16,f2022040100117.rev.6_6_001g.FDCC.GOES-16.txt,-9999,33.8289,-117.8737,10,49.7,38225.0,488.9,9325618,295.460,285.730,284.249,284.723,147.714,60.212,-9999,1
93866,6_6_001g,2022-02-09 08:51:17,GOES-16,FM?,GOES-16,GOES-16,f2022040085117.rev.6_6_001g.FDCC.GOES-16.txt,-9999,33.8289,-117.8737,10,152.8,25665.0,611.5,9325618,309.896,285.306,283.601,283.991,158.397,60.212,-9999,1


In [6]:
# join all GOES-16 dataframes into unified wfabba_goes_16_df
wfabba_goes_16_df = pd.concat([wfabba_goes_16_2019_df, wfabba_goes_16_2020_df, wfabba_goes_16_jan_2021_df, wfabba_goes_16_2021_df])
wfabba_goes_16_df["timestamp_converted"] = pd.to_datetime(wfabba_goes_16_df["Timestamp"], infer_datetime_format=True, origin="unix", utc=True)
wfabba_goes_16_df = wfabba_goes_16_df.reset_index()
wfabba_goes_16_df = wfabba_goes_16_df.drop(columns=["index"])
wfabba_goes_16_df

,Version,Timestamp,Satellite,FlightModel,ScanMode,ProductType,FileName,MissingValueCode,Latitude,Longitude,Code,FRP,Fire Size,Fire Temp,Pixel Size,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco,timestamp_converted
0,6_5_012g,2019-06-01 23:36:41,GOES-16,FM1,C,FDCC,f2019152233641.rev.6_5_012g.FDCC.GOES-16,-9999,34.1101,-117.2837,15,46.0,-9999.0,-9999.0,11478677,307.035,286.865,302.505,292.199,51.492,59.916,-9999,1,2019-06-01 23:36:41+00:00
1,6_5_012g,2019-06-01 23:36:41,GOES-16,FM1,C,FDCC,f2019152233641.rev.6_5_012g.FDCC.GOES-16,-9999,32.9690,-113.4948,10,52.6,50251.0,476.1,9680612,317.625,308.088,313.579,308.984,54.716,56.320,-9999,47,2019-06-01 23:36:41+00:00
2,6_5_012g,2019-06-01 20:01:41,GOES-16,FM1,C,FDCC,f2019152200141.rev.6_5_012g.FDCC.GOES-16,-9999,33.3033,-113.0092,15,12.9,-9999.0,-9999.0,9610866,324.175,308.601,323.940,314.648,13.367,56.164,-9999,51,2019-06-01 20:01:41+00:00
3,6_5_012g,2019-06-01 00:31:41,GOES-16,FM1,C,FDCC,f2019152003141.rev.6_5_012g.FDCC.GOES-16,-9999,34.8145,-113.5976,15,55.6,-9999.0,-9999.0,10216718,308.467,297.128,302.069,299.778,65.685,57.570,-9999,51,2019-06-01 00:31:41+00:00
4,6_5_012g,2019-06-01 00:31:41,GOES-16,FM1,C,FDCC,f2019152003141.rev.6_5_012g.FDCC.GOES-16,-9999,34.8947,-118.9655,15,71.9,-9999.0,-9999.0,12601857,301.172,281.546,292.217,289.893,61.326,61.659,-9999,91,2019-06-01 00:31:41+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300561,6_6_001g,2022-02-09 09:40:20,GOES-16,FM?,GOES-16,GOES-16,f2022040094020.rev.6_6_001g.FDCF.GOES-16.txt,-9999,33.8289,-117.8737,10,59.3,92471.0,447.4,9326390,296.811,285.730,283.625,283.939,151.404,60.212,-9999,1,2022-02-09 09:40:20+00:00
300562,6_6_001g,2022-02-09 09:40:20,GOES-16,FM?,GOES-16,GOES-16,f2022040094020.rev.6_6_001g.FDCF.GOES-16.txt,-9999,33.8030,-117.8540,10,68.2,269858.0,402.7,9316094,298.369,287.315,283.652,283.960,151.405,60.182,-9999,1,2022-02-09 09:40:20+00:00
300563,6_6_001g,2022-02-09 10:01:17,GOES-16,FM?,GOES-16,GOES-16,f2022040100117.rev.6_6_001g.FDCC.GOES-16.txt,-9999,33.8289,-117.8737,10,49.7,38225.0,488.9,9325618,295.460,285.730,284.249,284.723,147.714,60.212,-9999,1,2022-02-09 10:01:17+00:00
300564,6_6_001g,2022-02-09 08:51:17,GOES-16,FM?,GOES-16,GOES-16,f2022040085117.rev.6_6_001g.FDCC.GOES-16.txt,-9999,33.8289,-117.8737,10,152.8,25665.0,611.5,9325618,309.896,285.306,283.601,283.991,158.397,60.212,-9999,1,2022-02-09 08:51:17+00:00


In [7]:
#read in GOES 17 inputs
wfabba_goes_17_2019_df = pd.read_csv(processed_data_dir + "GOES-17-2019.csv")
wfabba_goes_17_2020_df = pd.read_csv(processed_data_dir + "GOES-17-2020.csv")
wfabba_goes_17_jan_2021_df = pd.read_csv(processed_data_dir + "GOES-17-Jan-2021.csv")
wfabba_goes_17_2021_df = pd.read_csv(processed_data_dir + "GOES-17-2021.csv")

In [8]:
#get rid of unnecessary columns including ones which contain the same values or all NaN
wfabba_goes_17_2019_df = wfabba_goes_17_2019_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_2020_df = wfabba_goes_17_2020_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_jan_2021_df = wfabba_goes_17_jan_2021_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"])
wfabba_goes_17_2021_df = wfabba_goes_17_2021_df.drop(columns = ["Unnamed: 0", "Algorithm","Instrument","DataSource", "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "Line", "Element"]) #2021 detections

In [9]:
# filter out any January data in wfabba_goes_17_2021_df since it already exists in wfabba_goes_17_jan_2021_df
print(len(wfabba_goes_17_2021_df))
wfabba_goes_17_2021_df = wfabba_goes_17_2021_df[wfabba_goes_17_2021_df["Timestamp"] >= "2021-02-01"]
wfabba_goes_17_2021_df = wfabba_goes_17_2021_df.reset_index()
wfabba_goes_17_2021_df = wfabba_goes_17_2021_df.drop(columns=["index"])
print(len(wfabba_goes_17_2021_df))
wfabba_goes_17_2021_df

379620
337812


,Version,Timestamp,Satellite,FlightModel,ScanMode,ProductType,FileName,MissingValueCode,Latitude,Longitude,Code,FRP,Fire Size,Fire Temp,Pixel Size,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco
0,6_6_001g,2021-02-01 20:26:17,GOES-17,FM?,GOES-17,GOES-17,f2021032202617.rev.6_6_001g.FDCC.GOES-17.txt,-9999,34.9345,-119.7105,15.0,19.8,-9999.0,-9999.0,6156361,303.602,277.521,299.613,291.022,52.227,44.873,-9999,51
1,6_6_001g,2021-02-01 20:25:25,GOES-17,FM?,GOES-17,GOES-17,f2021032202525.rev.6_6_001g.FDCM1.GOES-17.txt,-9999,34.9345,-119.7105,15.0,10.7,-9999.0,-9999.0,6156362,303.526,279.576,301.773,291.597,52.212,44.873,-9999,51
2,6_6_001g,2021-02-01 20:26:25,GOES-17,FM?,GOES-17,GOES-17,f2021032202625.rev.6_6_001g.FDCM1.GOES-17.txt,-9999,34.9345,-119.7105,15.0,11.5,-9999.0,-9999.0,6156362,303.639,278.675,301.692,291.601,52.227,44.873,-9999,51
3,6_6_001g,2021-02-01 20:24:25,GOES-17,FM?,GOES-17,GOES-17,f2021032202425.rev.6_6_001g.FDCM1.GOES-17.txt,-9999,34.9345,-119.7105,15.0,19.7,-9999.0,-9999.0,6156362,303.374,280.572,299.357,290.776,52.198,44.873,-9999,51
4,6_6_001g,2021-02-02 22:30:31,GOES-17,FM?,GOES-17,GOES-17,f2021033223031.rev.6_6_001g.FDCF.GOES-17.txt,-9999,34.9653,-118.0116,15.0,20.1,-9999.0,-9999.0,6260016,304.640,282.501,301.034,293.715,62.237,45.690,-9999,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337807,6_6_001g,2022-02-09 19:50:32,GOES-17,FM?,GOES-17,GOES-17,f2022040195032.rev.6_6_001g.FDCF.GOES-17.txt,-9999,33.9197,-114.4164,12.0,62.5,-9999.0,-9999.0,6380802,320.128,303.208,311.424,304.963,48.789,46.588,-9999,37
337808,6_6_001g,2022-02-09 19:50:32,GOES-17,FM?,GOES-17,GOES-17,f2022040195032.rev.6_6_001g.FDCF.GOES-17.txt,-9999,33.9207,-114.3908,10.0,77.6,110511.0,453.4,6381262,321.337,305.686,311.355,304.806,48.790,46.604,-9999,37
337809,6_6_001g,2022-02-09 20:50:32,GOES-17,FM?,GOES-17,GOES-17,f2022040205032.rev.6_6_001g.FDCF.GOES-17.txt,-9999,33.0108,-115.4984,10.0,62.0,105896.0,443.7,6190361,318.240,305.327,308.663,304.270,49.605,45.160,-9999,24
337810,6_6_001g,2022-02-09 20:50:32,GOES-17,FM?,GOES-17,GOES-17,f2022040205032.rev.6_6_001g.FDCF.GOES-17.txt,-9999,32.2436,-115.0157,10.0,48.5,55859.0,465.2,6135983,317.083,303.545,310.250,304.057,48.998,44.741,-9999,37


In [10]:
# join all GOES-17 dataframes into unified wfabba_goes_17_df
wfabba_goes_17_df = pd.concat([wfabba_goes_17_2019_df, wfabba_goes_17_2020_df, wfabba_goes_17_jan_2021_df, wfabba_goes_17_2021_df])
wfabba_goes_17_df["timestamp_converted"] = pd.to_datetime(wfabba_goes_17_df["Timestamp"], infer_datetime_format=True, origin="unix", utc=True)
wfabba_goes_17_df = wfabba_goes_17_df.reset_index()
wfabba_goes_17_df = wfabba_goes_17_df.drop(columns=["index"])
wfabba_goes_17_df

,Version,Timestamp,Satellite,FlightModel,ScanMode,ProductType,FileName,MissingValueCode,Latitude,Longitude,Code,FRP,Fire Size,Fire Temp,Pixel Size,Obs BT4,Obs BT11,Bkg BT4,Bkg BT11,SolZen,SatZen,RelAzi,Eco,timestamp_converted
0,6_5_012g,2019-06-01 18:58:57,GOES-16,FM2,M2,FDCM2,f2019152185857.rev.6_5_012g.FDCM2.GOES-17,-9999,32.5523,-114.9965,10.0,50.6,17180.0,548.7,6448748,323.734,311.935,317.761,311.218,13.606,45.030,-9999,24,2019-06-01 18:58:57+00:00
1,6_5_012g,2019-06-01 18:52:57,GOES-16,FM2,M2,FDCM2,f2019152185257.rev.6_5_012g.FDCM2.GOES-17,-9999,32.7453,-115.5775,15.0,61.8,-9999.0,-9999.0,6415740,324.352,307.847,316.439,308.990,14.963,44.874,-9999,37,2019-06-01 18:52:57+00:00
2,6_5_012g,2019-06-01 22:06:57,GOES-16,FM2,M2,FDCM2,f2019152220657.rev.6_5_012g.FDCM2.GOES-17,-9999,32.9714,-116.6118,15.0,9.8,-9999.0,-9999.0,6345534,303.715,288.028,302.233,295.853,33.402,44.514,-9999,22,2019-06-01 22:06:57+00:00
3,6_5_012g,2019-06-01 18:08:57,GOES-16,FM2,M2,FDCM2,f2019152180857.rev.6_5_012g.FDCM2.GOES-17,-9999,34.7783,-118.4323,12.0,440.7,-9999.0,-9999.0,6427847,351.035,307.493,316.979,307.734,25.775,45.308,-9999,51,2019-06-01 18:08:57+00:00
4,6_5_012g,2019-06-01 18:08:57,GOES-16,FM2,M2,FDCM2,f2019152180857.rev.6_5_012g.FDCM2.GOES-17,-9999,34.7791,-118.4075,10.0,464.5,2077.0,1476.8,6429915,353.142,308.442,316.979,307.734,25.757,45.321,-9999,51,2019-06-01 18:08:57+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1640537,6_6_001g,2022-02-09 19:50:32,GOES-17,FM?,GOES-17,GOES-17,f2022040195032.rev.6_6_001g.FDCF.GOES-17.txt,-9999,33.9197,-114.4164,12.0,62.5,-9999.0,-9999.0,6380802,320.128,303.208,311.424,304.963,48.789,46.588,-9999,37,2022-02-09 19:50:32+00:00
1640538,6_6_001g,2022-02-09 19:50:32,GOES-17,FM?,GOES-17,GOES-17,f2022040195032.rev.6_6_001g.FDCF.GOES-17.txt,-9999,33.9207,-114.3908,10.0,77.6,110511.0,453.4,6381262,321.337,305.686,311.355,304.806,48.790,46.604,-9999,37,2022-02-09 19:50:32+00:00
1640539,6_6_001g,2022-02-09 20:50:32,GOES-17,FM?,GOES-17,GOES-17,f2022040205032.rev.6_6_001g.FDCF.GOES-17.txt,-9999,33.0108,-115.4984,10.0,62.0,105896.0,443.7,6190361,318.240,305.327,308.663,304.270,49.605,45.160,-9999,24,2022-02-09 20:50:32+00:00
1640540,6_6_001g,2022-02-09 20:50:32,GOES-17,FM?,GOES-17,GOES-17,f2022040205032.rev.6_6_001g.FDCF.GOES-17.txt,-9999,32.2436,-115.0157,10.0,48.5,55859.0,465.2,6135983,317.083,303.545,310.250,304.057,48.998,44.741,-9999,37,2022-02-09 20:50:32+00:00


In [11]:
# filter out any low probability fire detections in both GOES-16 and GOES-17 WFABBA data
print(wfabba_goes_16_df.shape)
print(wfabba_goes_17_df.shape)

wfabba_goes_16_df = wfabba_goes_16_df[(wfabba_goes_16_df["Code"] != 15) & (wfabba_goes_16_df["Code"] != 35)]
wfabba_goes_17_df = wfabba_goes_17_df[(wfabba_goes_17_df["Code"] != 15) & (wfabba_goes_17_df["Code"] != 35)]

print(wfabba_goes_16_df.shape)
print(wfabba_goes_17_df.shape)

(300566, 24)
(1640542, 24)
(184404, 24)
(883295, 24)


### Convert the coordinates of WFABBA GOES-17 and GOES-16 from EPSG 4326 to EPSG 3310 to allow for distance calculations down to the meter

In [12]:
#convert WFABBA GOES 16 coordinates from EPSG 4326 to EPSG 3310
coords = [Point(xy) for xy in zip(wfabba_goes_16_df['Longitude'], wfabba_goes_16_df['Latitude'])]
wfabba_goes_16_df = GeoDataFrame(wfabba_goes_16_df, crs = "EPSG:4326", geometry = coords) 
wfabba_goes_16_df = wfabba_goes_16_df.to_crs('EPSG:3310')
wfabba_goes_16_df[["Latitude","Longitude","geometry"]]

,Latitude,Longitude,geometry
1,32.9690,-113.4948,POINT (608345.835 -539578.549)
5,32.9941,-113.5104,POINT (606699.070 -536904.408)
6,32.9921,-113.4776,POINT (609775.877 -536915.028)
7,32.9710,-113.5276,POINT (605268.147 -539567.438)
8,32.9690,-113.4948,POINT (608345.835 -539578.549)
...,...,...,...
300560,32.8708,-115.4607,POINT (425192.645 -561154.402)
300561,33.8289,-117.8737,POINT (196846.871 -462936.505)
300562,33.8030,-117.8540,POINT (198735.340 -465766.968)
300563,33.8289,-117.8737,POINT (196846.871 -462936.505)


In [13]:
#convert WFABBA GOES 17 coordinates from EPSG 4326 to EPSG 3310
coords = [Point(xy) for xy in zip(wfabba_goes_17_df['Longitude'], wfabba_goes_17_df['Latitude'])]
wfabba_goes_17_df = GeoDataFrame(wfabba_goes_17_df, crs = "EPSG:4326", geometry = coords) 
wfabba_goes_17_df = wfabba_goes_17_df.to_crs('EPSG:3310')
wfabba_goes_17_df[["Latitude","Longitude","geometry"]]

,Latitude,Longitude,geometry
0,32.5523,-114.9965,POINT (470495.929 -594187.763)
3,34.7783,-118.4323,POINT (143396.998 -358615.563)
4,34.7791,-118.4075,POINT (145663.743 -358488.961)
5,34.7335,-118.2973,POINT (155833.437 -363374.892)
6,34.7086,-118.3041,POINT (155260.597 -366149.588)
...,...,...,...
1640537,33.9197,-114.4164,POINT (516065.792 -439872.805)
1640538,33.9207,-114.3908,POINT (518422.566 -439622.421)
1640539,33.0108,-115.4984,POINT (420927.893 -545831.632)
1640540,32.2436,-115.0157,POINT (470487.696 -628405.154)


## 2) Camera Metadata Processing

In [14]:
# read in camera metadata
camera_metadata_df = pd.read_csv("../../data/processed/camera_metadata_hpwren.csv")
camera_metadata_df

,camera_id,direction,camera_name,image_id,long,lat,elevation,geometry.type,geometry.coordinates,x_resolution,y_resolution,center_lat,center_long,properties.description.url
0,hpwren0_unknown direction,unknown direction,NaN,tje-1-mobo-c,-117.120000,32.550000,10.0,Point,"[-117.12, 32.55, 10]",NaN,NaN,NaN,NaN,http://hpwren.ucsd.edu/cameras/TJE.html
1,hpwren1_north,north,Big Black Mountain,bm-n-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.181599,-116.807554,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
2,hpwren1_east,east,Big Black Mountain,bm-e-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.158781,-116.790230,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
3,hpwren1_south,south,Big Black Mountain,bm-s-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.157932,-116.807962,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
4,hpwren1_west,west,Big Black Mountain,bm-w-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.159091,-116.858706,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,hpwren_missing5_west,west,Santiago Peak,stgo-w-mobo-c,-117.534115,33.711172,5669.0,NaN,NaN,3072.0,2048.0,33.710503,-117.554501,NaN
127,hpwren_missing6_north,north,White Star,ws-n-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,32.650875,-116.317954,NaN
128,hpwren_missing6_east,east,White Star,ws-e-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,32.646405,-116.305941,NaN
129,hpwren_missing6_south,south,White Star,ws-s-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,32.644742,-116.317985,NaN


## 3) Matching WFABBA to SmokeyNet 

In [15]:
#Create dataframe for every minute of specified time period
times = []
start = datetime(2019, 6 , 1, 0, 0, 0, 0, pytz.UTC)
end = datetime(2021, 7, 11, 23, 59, 0, 0, pytz.UTC)

while start <= end:
    times.append(start)
    start = start + timedelta(minutes = 1)

minutes_df = pd.DataFrame(times, columns = ["timestamp"])
minutes_df

,timestamp
0,2019-06-01 00:00:00+00:00
1,2019-06-01 00:01:00+00:00
2,2019-06-01 00:02:00+00:00
3,2019-06-01 00:03:00+00:00
4,2019-06-01 00:04:00+00:00
...,...
1111675,2021-07-11 23:55:00+00:00
1111676,2021-07-11 23:56:00+00:00
1111677,2021-07-11 23:57:00+00:00
1111678,2021-07-11 23:58:00+00:00


In [16]:
# Create testing SmokeyNet df
df_test = pd.read_json(raw_data_dir + "smokeynet_test.json", orient="index").reset_index().rename(columns={"index":"filepath"})
df_test["type"] = "test"
df_test 

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test
...,...,...,...,...,...,...,...
4880,20161113_FIRE_bm-n-mobo-c/1479069033_+02100,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test
4881,20161113_FIRE_bm-n-mobo-c/1479069093_+02160,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test
4882,20161113_FIRE_bm-n-mobo-c/1479069153_+02220,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test
4883,20161113_FIRE_bm-n-mobo-c/1479069213_+02280,20161113_FIRE_bm-n-mobo-c,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",test


In [17]:
#Create validating SmokeyNet df
df_valid = pd.read_json(raw_data_dir + "smokeynet_valid.json", orient="index").reset_index().rename(columns={"index":"filepath"})
df_valid["type"] = "valid"
df_valid

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type
0,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
1,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
2,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
3,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4,20200807_AppleFire-backfire-operation_hp-n-mob...,20200807_AppleFire-backfire-operation_hp-n-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
...,...,...,...,...,...,...,...
4906,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4907,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4908,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
4909,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid


In [18]:
#Join the SmokeyNet DFs together. For now just joining validation and test DFs
df_labels = pd.concat([df_test, df_valid]).reset_index().drop(columns = ["index"])
df_labels

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test
...,...,...,...,...,...,...,...
9791,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
9792,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
9793,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid
9794,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid


In [19]:
# set the date and year columns
df_labels["date"] = df_labels["camera_name"].str.split("_", n=1, expand=True)[0]
df_labels["year"] = df_labels["date"].str[:4]
df_labels["date"] = pd.to_datetime(df_labels["date"])
df_labels

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019
...,...,...,...,...,...,...,...,...,...
9791,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
9792,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
9793,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
9794,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020


In [20]:
# keeping only entries from 2019-06-01 onwards
df_labels_filtered = df_labels[df_labels["date"] >= "2019-06-01"].reset_index().drop(columns=["index"])
df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,20191001_FIRE_lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019
...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,20200813_Ranch2Fire_marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020


In [21]:
#create time, datetime, event_name, camera_name attributes
df_labels_filtered["time"] = df_labels_filtered["filepath"].str.split("/").str[1]
df_labels_filtered["time"] = df_labels_filtered["time"].str.split("_").str[0]
df_labels_filtered["datetime"] = pd.to_datetime(df_labels_filtered["time"], unit="s", origin="unix", utc=True)
df_labels_filtered["event_name"] = df_labels_filtered["filepath"].str.split("/").str[0]
df_labels_filtered["camera_name"] = df_labels_filtered["camera_name"].str.split("_").str[-1]

#Edge case: 2 camera names not following naming convention. Uses "-" instead of "_"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "FIRE-pi-s-mobo-c", ["camera_name"]] = "pi-s-mobo-c"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "FIRE-smer-tcs8-mobo-c", ["camera_name"]] = "smer-tcs8-mobo-c"

#some of the camera names are using old naming standards and may not immediately match to the metadata
#manually setting these camera names to ones listed in the metadata
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "lo-s-mobo-c", ["camera_name"]] = "ml-s-mobo-c"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "om-w-mobo", ["camera_name"]] = "om-w-mobo-c"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "so-w-mobo-c", ["camera_name"]] = "sojr-w-mobo-c"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "wc-e-mobo-c", ["camera_name"]] = "wc-e-mobo-c-deg"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "wc-n-mobo-c", ["camera_name"]] = "wc-n-mobo-c-deg"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "wc-s-mobo-c", ["camera_name"]] = "wc-s-mobo-c-deg"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "pi-s-mobo", ["camera_name"]] = "pi-s-mobo-c"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "om-s-mobo", ["camera_name"]] = "om-s-mobo-c"
df_labels_filtered.loc[df_labels_filtered["camera_name"] == "lp-s-mobo", ["camera_name"]] = "lp-s-mobo-c"

df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year,time,datetime,event_name
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950465,2019-10-01 17:21:05+00:00,20191001_FIRE_lp-s-mobo-c
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950525,2019-10-01 17:22:05+00:00,20191001_FIRE_lp-s-mobo-c
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950585,2019-10-01 17:23:05+00:00,20191001_FIRE_lp-s-mobo-c
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950645,2019-10-01 17:24:05+00:00,20191001_FIRE_lp-s-mobo-c
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019,1569950705,2019-10-01 17:25:05+00:00,20191001_FIRE_lp-s-mobo-c
...,...,...,...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360042,2020-08-13 23:07:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360098,2020-08-13 23:08:18+00:00,20200813_Ranch2Fire_marconi-n-mobo-c
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360162,2020-08-13 23:09:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360219,2020-08-13 23:10:19+00:00,20200813_Ranch2Fire_marconi-n-mobo-c


In [22]:
# join SmokeyNet data with camera metadata
df_labels_filtered = df_labels_filtered.merge(camera_metadata_df, left_on="camera_name", right_on="image_id", how="left")
df_labels_filtered = df_labels_filtered.drop(columns=["camera_name_y", "image_id", "elevation", "geometry.type", "geometry.coordinates", "x_resolution", "y_resolution", "center_lat", "center_long", "properties.description.url"])
df_labels_filtered = df_labels_filtered.rename(columns = {"camera_name_x":"camera_name"})
df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year,time,datetime,event_name,camera_id,direction,long,lat
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950465,2019-10-01 17:21:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950525,2019-10-01 17:22:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950585,2019-10-01 17:23:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950645,2019-10-01 17:24:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019,1569950705,2019-10-01 17:25:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360042,2020-08-13 23:07:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360098,2020-08-13 23:08:18+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360162,2020-08-13 23:09:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360219,2020-08-13 23:10:19+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127


In [23]:
# convert joined SmokeyNet-camera metadata dataframe's coordinates from EPSG 4326 to EPSG 3310
coords = [Point(xy) for xy in zip(df_labels_filtered['long'], df_labels_filtered['lat'])]
df_labels_filtered = GeoDataFrame(df_labels_filtered, crs = "EPSG:4326", geometry = coords) 
df_labels_filtered = df_labels_filtered.to_crs('EPSG:3310')
df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year,time,datetime,event_name,camera_id,direction,long,lat,geometry
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950465,2019-10-01 17:21:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517,POINT (303757.711 -584899.981)
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950525,2019-10-01 17:22:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517,POINT (303757.711 -584899.981)
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950585,2019-10-01 17:23:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517,POINT (303757.711 -584899.981)
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950645,2019-10-01 17:24:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517,POINT (303757.711 -584899.981)
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019,1569950705,2019-10-01 17:25:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517,POINT (303757.711 -584899.981)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360042,2020-08-13 23:07:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127,POINT (234115.176 -496381.326)
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360098,2020-08-13 23:08:18+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127,POINT (234115.176 -496381.326)
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360162,2020-08-13 23:09:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127,POINT (234115.176 -496381.326)
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360219,2020-08-13 23:10:19+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127,POINT (234115.176 -496381.326)


In [24]:
# rounds timestamps to nearest minute on the dot
def round_secs(x):
    x = x + timedelta(minutes = 1)
    x = x.replace(second=0)
    return x

In [25]:
# determines if a WFABBA detection is within the same direction as the camera
def is_in_camera_direction(camera_geometry_pt, direction, wfabba_geometry_pt):
    if direction == "north":
        return wfabba_geometry_pt.y >= camera_geometry_pt.y
    elif direction == "south":
        return wfabba_geometry_pt.y <= camera_geometry_pt.y
    elif direction == "east":
        return wfabba_geometry_pt.x >= camera_geometry_pt.x
    elif direction == "west":
        return wfabba_geometry_pt.x <= camera_geometry_pt.x
    else:
        # unknown or something else
        pass
    

In [26]:
# finds any matches with specified WFABBA dataset based off of 
# whether distance to camera is within specified radius & camera direction
def matches_distance_prox(camera_geometry, direction, radius_miles, wfabba_df):    
    wfabba_df["distance_m"] = wfabba_df["geometry"].distance(camera_geometry)
    wfabba_df["distance_mi"] = wfabba_df["distance_m"]/1609.344        
    match_results_df = wfabba_df[(wfabba_df["distance_mi"] <= radius_miles)].copy()
    
    #filter for detections within same direction
    match_results_df["is_in_direction"] = match_results_df.apply(
        lambda row: is_in_camera_direction(camera_geometry, direction, row["geometry"]), axis=1
    )
    match_results_df = match_results_df[match_results_df["is_in_direction"] == True]

    return match_results_df

In [27]:
# round the SmokeyNet timestamps to nearest minute on the dot
df_labels_filtered["datetime_rounded"] = df_labels_filtered["datetime"].apply(lambda x: round_secs(x))
df_labels_filtered

,filepath,camera_name,image_gt,tile_gt,image_pred,tile_pred,type,date,year,time,datetime,event_name,camera_id,direction,long,lat,geometry,datetime_rounded
0,20191001_FIRE_lp-s-mobo-c/1569950465_-02281,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950465,2019-10-01 17:21:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517,POINT (303757.711 -584899.981),2019-10-01 17:22:00+00:00
1,20191001_FIRE_lp-s-mobo-c/1569950525_-02221,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950525,2019-10-01 17:22:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517,POINT (303757.711 -584899.981),2019-10-01 17:23:00+00:00
2,20191001_FIRE_lp-s-mobo-c/1569950585_-02161,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950585,2019-10-01 17:23:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517,POINT (303757.711 -584899.981),2019-10-01 17:24:00+00:00
3,20191001_FIRE_lp-s-mobo-c/1569950645_-02101,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",test,2019-10-01,2019,1569950645,2019-10-01 17:24:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517,POINT (303757.711 -584899.981),2019-10-01 17:25:00+00:00
4,20191001_FIRE_lp-s-mobo-c/1569950705_-02041,lp-s-mobo-c,0,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",test,2019-10-01,2019,1569950705,2019-10-01 17:25:05+00:00,20191001_FIRE_lp-s-mobo-c,hpwren12_south,south,-116.764561,32.701517,POINT (303757.711 -584899.981),2019-10-01 17:26:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8820,20200813_Ranch2Fire_marconi-n-mobo-c/159736004...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360042,2020-08-13 23:07:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127,POINT (234115.176 -496381.326),2020-08-13 23:08:00+00:00
8821,20200813_Ranch2Fire_marconi-n-mobo-c/159736009...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360098,2020-08-13 23:08:18+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127,POINT (234115.176 -496381.326),2020-08-13 23:09:00+00:00
8822,20200813_Ranch2Fire_marconi-n-mobo-c/159736016...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360162,2020-08-13 23:09:22+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127,POINT (234115.176 -496381.326),2020-08-13 23:10:00+00:00
8823,20200813_Ranch2Fire_marconi-n-mobo-c/159736021...,marconi-n-mobo-c,1,[],0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",valid,2020-08-13,2020,1597360219,2020-08-13 23:10:19+00:00,20200813_Ranch2Fire_marconi-n-mobo-c,hpwren30_north,north,-117.480907,33.519127,POINT (234115.176 -496381.326),2020-08-13 23:11:00+00:00


In [28]:
# get all unique cameras being considered
unique_cameras = df_labels_filtered["camera_name"].unique()
unique_cameras

array(['lp-s-mobo-c', 'pi-s-mobo-c', 'pi-n-mobo-c', 'ml-w-mobo-c',
       'ml-s-mobo-c', 'om-e-mobo-c', 'lp-n-mobo-c', 'mlo-s-mobo-c',
       'bh-w-mobo-c', 'sm-e-mobo-c', 'sm-s-mobo-c', 'om-w-mobo-c',
       'om-s-mobo-c', 'bm-w-mobo-c', 'sojr-w-mobo-c', 'hp-e-mobo-c',
       'dwpgm-n-mobo-c', 'hp-n-mobo-c', 'rm-e-mobo-c', 'cp-s-mobo-c',
       'tp-s-mobo-c', 'smer-tcs8-mobo-c', 'wc-e-mobo-c-deg',
       'wc-n-mobo-c-deg', 'rm-w-mobo-c', 'smer-tcs9-mobo-c',
       'wc-s-mobo-c-deg', 'sclm-e-mobo-c', 'mg-n-mobo-c', 'lp-w-mobo-c',
       'hp-s-mobo-c', 'mlo-n-mobo-c', 'pi-e-mobo-c', 'om-n-mobo-c',
       'sp-n-mobo-c', 'sm-w-mobo-c', 'rm-n-mobo-c', 'hp-w-mobo-c',
       'tp-w-mobo-c', '69bravo-e-mobo-c', 'pi-w-mobo-c', 'sjh-n-mobo-c',
       'bl-n-mobo-c', 'syp-w-mobo-c', 'sm-n-mobo-c', 'vo-n-mobo-c',
       'lp-e-mobo-c', 'bm-e-mobo-c', 'bl-s-mobo-c', 'marconi-n-mobo-c'],
      dtype=object)

In [29]:
# if there are cameras that don't have associated directions, filter them out
unusable_cameras = df_labels_filtered[df_labels_filtered["direction"].isna()]["camera_name"].unique()
unique_cameras = np.setdiff1d(unique_cameras, unusable_cameras)
unique_cameras 

array(['69bravo-e-mobo-c', 'bh-w-mobo-c', 'bl-n-mobo-c', 'bl-s-mobo-c',
       'bm-e-mobo-c', 'bm-w-mobo-c', 'cp-s-mobo-c', 'dwpgm-n-mobo-c',
       'hp-e-mobo-c', 'hp-n-mobo-c', 'hp-s-mobo-c', 'hp-w-mobo-c',
       'lp-e-mobo-c', 'lp-n-mobo-c', 'lp-s-mobo-c', 'lp-w-mobo-c',
       'marconi-n-mobo-c', 'mg-n-mobo-c', 'ml-s-mobo-c', 'ml-w-mobo-c',
       'mlo-n-mobo-c', 'mlo-s-mobo-c', 'om-e-mobo-c', 'om-n-mobo-c',
       'om-s-mobo-c', 'om-w-mobo-c', 'pi-e-mobo-c', 'pi-n-mobo-c',
       'pi-s-mobo-c', 'pi-w-mobo-c', 'rm-e-mobo-c', 'rm-n-mobo-c',
       'rm-w-mobo-c', 'sclm-e-mobo-c', 'sjh-n-mobo-c', 'sm-e-mobo-c',
       'sm-n-mobo-c', 'sm-s-mobo-c', 'sm-w-mobo-c', 'sojr-w-mobo-c',
       'syp-w-mobo-c', 'tp-s-mobo-c', 'tp-w-mobo-c', 'vo-n-mobo-c',
       'wc-e-mobo-c-deg', 'wc-n-mobo-c-deg', 'wc-s-mobo-c-deg'],
      dtype=object)

### Find all potential WFABBA GOES-16 and GOES-17 matches for SmokeyNet predictions for each camera station
Loop through each camera station. For each camera station's SmokeyNet predictions, match with potential WFABBA GOES-16 and WFABBA GOES-17 detections that are within a specified radius (default value of 35 miles), the same camera direction, and the same minute. Join these potentially matched SmokeyNet, WFABBA GOES-16, and WFABBA GOES-17 predictions together. To determine hard voting ensemble final prediction for each grouping of detections, add up how many individual models detected smoke, with a majority vote being declared a positive prediction. Then output the camera station's predictions into a csv file for future accuracy evaluation.

In [30]:
%%time

csv_suffix = "_all_hard_voting_35.csv"

# spatial radius of potential WFABBA matches
distance_miles = 35

#looping for each camera station
for camera in unique_cameras:
    
    print("Camera:",camera)
    camera_df = df_labels_filtered[df_labels_filtered["camera_name"].str.contains(camera)].copy()

    camera_instance = camera_df.iloc[0]
    
    #Find GOES-16 matches
    goes_16_dist_match_df = matches_distance_prox(camera_instance["geometry"], camera_instance["direction"], distance_miles, wfabba_goes_16_df)
    goes_16_dist_match_df["timestamp_converted_rounded"] = goes_16_dist_match_df["timestamp_converted"].apply(lambda x: round_secs(x))
    goes_16_dist_match_df = goes_16_dist_match_df.drop_duplicates(subset = ["timestamp_converted_rounded"])

    #Find GOES-17 matches
    goes_17_dist_match_df = matches_distance_prox(camera_instance["geometry"], camera_instance["direction"], distance_miles, wfabba_goes_17_df)
    goes_17_dist_match_df["timestamp_converted_rounded"] = goes_17_dist_match_df["timestamp_converted"].apply(lambda x: round_secs(x))
    goes_17_dist_match_df = goes_17_dist_match_df.drop_duplicates(subset = ["timestamp_converted_rounded"])

    #SmokeyNet_join
    test_df = minutes_df.merge(camera_df, left_on = "timestamp", right_on = "datetime_rounded",how="left")
    test_df = test_df.rename(columns = {"geometry":"HPWREN_Station_geometry", "lat":"HPWREN_lat", "long":"HPWREN_long", "datetime_rounded":"SmokeyNet_datetime_rounded"})
    print("joined SmokeyNet")
    
    #GOES-16 Join
    test_df = test_df.merge(goes_16_dist_match_df[["timestamp_converted_rounded", "geometry"]], left_on = "timestamp", right_on = "timestamp_converted_rounded",how="left")
    test_df = test_df.rename(columns = {"geometry":"WFABBA_GOES16_geometry", "timestamp_converted_rounded":"WFABBA_GOES16_timestamp_converted_rounded"})
    test_df = test_df[["timestamp","camera_name", "image_gt", "image_pred", "type", "WFABBA_GOES16_geometry"]]
    test_df.loc[test_df["WFABBA_GOES16_geometry"] != None,'goes16_pred'] = 1
    test_df.loc[test_df["WFABBA_GOES16_geometry"] == None,'goes16_pred'] = 0
    print("joined GOES16")

    #GOES-17 Join
    test_df = test_df.merge(goes_17_dist_match_df[["timestamp_converted_rounded", "geometry"]], left_on = "timestamp", right_on = "timestamp_converted_rounded",how="left")
    test_df = test_df.rename(columns = {"geometry":"WFABBA_GOES17_geometry", "timestamp_converted_rounded":"WFABBA_GOES17_timestamp_converted_rounded"})
    test_df = test_df[["timestamp","camera_name", "image_gt", "image_pred", "type", "WFABBA_GOES16_geometry", "goes16_pred", "WFABBA_GOES17_geometry"]]
    test_df.loc[test_df["WFABBA_GOES17_geometry"] != None,'goes17_pred'] = 1
    test_df.loc[test_df["WFABBA_GOES17_geometry"] == None,'goes17_pred'] = 0
    print("joined GOES17")

    #Get all votes and determine if smoke was detected by majority rule
    test_df["final_vote"] = test_df["image_pred"] + test_df["goes16_pred"] + test_df["goes17_pred"]
    test_df.loc[test_df["final_vote"] >= 2,'final_pred'] = 1
    test_df.loc[test_df["final_vote"] < 2,'final_pred'] = 0

    image_labels = test_df[~test_df["image_gt"].isna()]["image_gt"]
    smokeynet_preds = test_df[~test_df["image_gt"].isna()]["image_pred"]
    ensemble_preds = test_df[~test_df["image_gt"].isna()]["final_pred"]

    baseline_score = accuracy_score(image_labels, smokeynet_preds)
    ensemble_score = accuracy_score(image_labels, ensemble_preds)
    
    print("Baseline score:", baseline_score)
    print("Ensemble score:", ensemble_score)
    test_df[~test_df["image_gt"].isna()][["timestamp","image_gt", "image_pred", "goes16_pred", "goes17_pred", "final_pred","type"]]\
        .to_csv(processed_data_dir + camera + csv_suffix)
    print("=====================================================")

Camera: 69bravo-e-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.9625
Ensemble score: 0.4875
Camera: bh-w-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.9617834394904459
Ensemble score: 0.7006369426751592
Camera: bl-n-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.75
Ensemble score: 0.4875
Camera: bl-s-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.9230769230769231
Ensemble score: 0.6282051282051282
Camera: bm-e-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.9358974358974359
Ensemble score: 0.5
Camera: bm-w-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.8625
Ensemble score: 0.4875
Camera: cp-s-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.9113924050632911
Ensemble score: 0.7531645569620253
Camera: dwpgm-n-mobo-c
joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.975
Ensemble score: 0.5
Camera: hp-e-mobo-c
joined Sm

joined SmokeyNet
joined GOES16
joined GOES17
Baseline score: 0.7449664429530202
Ensemble score: 0.6241610738255033
CPU times: user 13min 43s, sys: 23.5 s, total: 14min 7s
Wall time: 14min 7s
